In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [2]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="2400bef4-fcef-4604-9496-195b7a0935e6",
    resource_group_name="jetson-rg",
    workspace_name="jetson-aml",
)

In [3]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print( f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is.")

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=1,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="LowPriority",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()

print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
)

You already have a cluster named cpu-cluster, we'll reuse it as is.
AMLCompute with name cpu-cluster is created, the compute size is STANDARD_DS3_V2


In [4]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="akhayash/donkey45:latest",
    name="mydonkey45",
    description="Donkey Car 4.5 environment on Ubuntu 18.04",
)
env = ml_client.environments.create_or_update(env_docker_image)

print(env)


creation_context:
  created_at: '2023-12-26T14:12:50.676111+00:00'
  created_by: Akihito Hayashi
  created_by_type: User
  last_modified_at: '2023-12-26T14:12:50.676111+00:00'
  last_modified_by: Akihito Hayashi
  last_modified_by_type: User
description: Donkey Car 4.5 environment on Ubuntu 18.04
id: azureml:/subscriptions/2400bef4-fcef-4604-9496-195b7a0935e6/resourceGroups/jetson-rg/providers/Microsoft.MachineLearningServices/workspaces/jetson-aml/environments/mydonkey45/versions/3
image: akhayash/donkey45:latest
name: mydonkey45
os_type: linux
tags: {}
version: '3'



In [6]:
import mlflow
import logging

logging.getLogger("azure").setLevel(logging.DEBUG)

mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
mlflow.set_tracking_uri(mlflow_tracking_uri)

experiment_name = 'donkey45'
mlflow.set_experiment(experiment_name)

2023/12/26 23:17:54 INFO mlflow.tracking.fluent: Experiment with name 'donkey45' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1703600276377, experiment_id='e0575828-b43b-4b40-8da1-a8159c8e4e52', last_update_time=None, lifecycle_stage='active', name='donkey45', tags={}>

In [11]:

# run_name should be unique with "donkey45" and yyyy-mm-dd:hh:mm:ss format based on execution time
mlflow.start_run(run_name=run_name)

# Your code

mlflow.end_run()